#Programa para el cálculo de uso codónico

Autor: Esteban Velásquez

tiene el objetivo de calcular la frecuencia
del uso de los codones de un genoma (por ejemplo: cpDNA, mtDNA, Genoma
bacteriano, etc), se requiere desarrollar un programa que reciba de entrada un
archivo Genbank (.gb, ej: NC_016736.1) y como salida genera una tabla (.tsv) con el usage codon bias.
Es necesario seleccionar entre los sigueintes códigos genéticos:

1. The Standard Code
2. The Vertebrate Mitochondrial Code
3. The Yeast Mitochondrial Code
4. The Mold, Protozoan, and Coelenterate Mitochondrial Code and the Mycoplasma/Spiroplasma Code
5. The Invertebrate Mitochondrial Code
6. The Ciliate, Dasycladacean and Hexamita Nuclear Code
9. The Echinoderm and Flatworm Mitochondrial Code
10. The Euplotid Nuclear Code
11. The Bacterial, Archaeal and Plant Plastid Code
12. The Alternative Yeast Nuclear Code
13. The Ascidian Mitochondrial Code
14. The Alternative Flatworm Mitochondrial Code
16. Chlorophycean Mitochondrial Code
21. Trematode Mitochondrial Code
22. Scenedesmus obliquus Mitochondrial Code
23. Thraustochytrium Mitochondrial Code
24. Pterobranchia Mitochondrial Code
25. Candidate Division SR1 and Gracilibacteria Code
26. Pachysolen tannophilus Nuclear Code
27. Karyorelict Nuclear Code
28. Condylostoma Nuclear Code
29. Mesodinium Nuclear Code
30. Peritrich Nuclear Code
31. Blastocrithidia Nuclear Code
33. Cephalodiscidae Mitochondrial UAA-Tyr Code



Funciones secundarias necesarias para el main:


In [33]:
def DNA(correo,ID):#función que me obtiene el codigo genetico de NCBI
    import Bio 
    from Bio import Entrez , SeqIO
    from Bio.Seq import Seq
    Entrez.email = correo
    handle = Entrez.efetch(db = "nucleotide", id = ID, rettype = "gb", retmode = "text") 
    genomas=SeqIO.read(handle,"genbank")
    return genomas

In [34]:
def ADNcodificantealt(genoma):#función alternapara sacar ADN codificante pero que no discrimina exones pero si es general y no depende de NCBI
    import Bio #primero se inicializa biopython para luego abrir el genoma correspondiente
    import numpy as np
    from Bio import Entrez , SeqIO
    from Bio.Data import CodonTable
    Startcod=CodonTable.unambiguous_dna_by_id[gencod].start_codons
    Endcod=CodonTable.unambiguous_dna_by_id[gencod].stop_codons
    from Bio.Seq import Seq
    ADNcodificante=Seq("")
    c=0
    for i in range(0,len(genoma)-2):
        for r in range(0,len(Startcod)):
            if ((genoma[i]+genoma[i+1]+genoma[i+2] == Startcod[r])&(c==0)):
                b=i
                c=1
        for v in range(0,len(Endcod)):
            if ((genoma[i]+genoma[i+1]+genoma[i+2] == Endcod[v])&(c==1)):
                c=0
                k=i
                string2=(genoma[b:(k+1)])
                ADNcodificante=ADNcodificante+string2                    

In [35]:
def ADNcodificante(genomas):#función que me saca el ADNcodificante de un NCBI y no de todos lamentablemente
    import Bio
    from Bio import Entrez , SeqIO
    from Bio.Seq import Seq
    jct_info=""
    for f in genomas.features:
        if f.type == 'CDS':
            jct_info=jct_info+str(f.location)
    import re
    jctP = re.compile("\[\d+\:\d+\]")   
    jcts = jctP.findall(jct_info)
    ADNcod=Seq("")
    for i in range(len(jcts)):
        ini=int(jcts[i].strip("[").strip("]").split(":")[0])
        fin=int(jcts[i].strip("[").strip("]").split(":")[1])
        ADNcod=ADNcod+(genomas.seq[ini:fin])
    return ADNcod

In [36]:
def transcribir(ADNcodificante):
    mRNAcodificante=ADNcodificante.transcribe()
    return mRNAcodificante

In [37]:
def codones(mARNcod):
    setcodones=set()
    leng=len(mARNcod)
    for i in range(leng-2):
        a=mARNcod[i]
        b=mARNcod[i+1]
        c=mARNcod[i+2]
        setcodones.add(a+b+c)
    return setcodones

In [48]:
def listarcod(setcod):
    listcod=list(setcod)
    return listcod

In [50]:
def traducirlista(listcod):
    import Bio
    from Bio import Entrez , SeqIO
    from Bio.Seq import Seq
    amino= [0 for x in range (len(listcod))]
    for n in range(0,(len(listcod))):
        amino[n]=(Seq(listcod[n]).translate(table=gencode)[0])
    return amino

In [40]:
def contarcodones(mARNcod,listcod):
    numcod=[0 for x in range (len(listcod))]
    long=len(listcod)
    leng=len(mARNcod)
    for i in range(0,(leng-2)):
        for n in range(0,(long)):
            if mARNcod[i]+mARNcod[i+1]+mARNcod[i+2]==listcod[n]:
                numcod[n]=numcod[n]+1
    return numcod

In [41]:
def totporaminoacido (amino,numcod):
    totalaminoacido=[0 for x in range(len(amino))]
    for i in range (0,(len(amino))):
        for j in range (0,(len(amino))):
            if amino[i] == amino[j]:
                totalaminoacido[i]=totalaminoacido[i]+numcod[j]
    return totalaminoacido       

In [42]:
def cuantosporaminoacido(amino):
    cuantosporaa=[0 for x in range(len(amino))]
    for i in range (0,(len(amino))):
        for j in range (0,(len(amino))):
            if amino[i] == amino[j]:
                cuantosporaa[i]=cuantosporaa[i]+1
    return cuantosporaa

In [53]:
def calcularRSCU(numcod,totalaminoacido,cuantosporaa):
    RSCU=[0 for x in range(len(numcod))]
    for i in range(0,(len(numcod))):
        RSCU[i]=round((numcod[i]/(totalaminoacido[i]/cuantosporaa[i])),3)
    return RSCU

In [55]:
def calcporcentaje(numcod,totalaminoacido):
    porcentaje=[0 for x in range(len(numcod))]
    for i in range (0,(len(numcod))):
        porcentaje[i]=round(((numcod[i]/totalaminoacido[i])*100),2)
    return porcentaje

In [45]:
def TRNA(genomas,listcod):#función que saca los TRNA que son codificados en el genoma
    import Bio
    from Bio import Entrez , SeqIO
    from Bio.Seq import Seq
    settrna=set()
    for i in range(0,len(genomas.features)):
        if("gene" in genomas.features[i].qualifiers):
            if(genomas.features[i].qualifiers['gene'][0][0]+genomas.features[i].qualifiers['gene'][0][1]+genomas.features[i].qualifiers['gene'][0][2]=="trn"):
                settrna.add((genomas.features[i].qualifiers['gene'][0]))
    genomas.features[i].qualifiers["gene"][0]
    listtrna=list(settrna)
    trna=[0 for x in range(len(listcod))]
    for i in range(0,len(listtrna)):
        for j in range(0,len(listcod)):
            if (Seq(listtrna[i].split("-")[1]).reverse_complement()==listcod[j]):
                trna[j]=listtrna[i]
    return trna



In [46]:
def creartabla(listcod,amino,numcod,RSCU,porcentaje,tRNA):
    import pandas as pd
    tabla = pd.DataFrame([listcod,amino,numcod,RSCU,porcentaje,tRNA]).T
    tabla.columns=['Codon','Amino acid','Number','RSCU','%','tRNA']
    tabla.to_csv('tablacodonusage.tsv', sep='\t', index=False)
    return tabla

FUNCION PRINCIPAL

In [56]:
def codonusagebias(ID,correo,gencode):
    """
    MAIN
    """
    genomas=DNA(correo,ID)
    ADNcod=ADNcodificante(genomas)
    mARNcod=transcribir(ADNcod)
    setcod=codones(mARNcod)
    listcod=listarcod(setcod)
    amino=traducirlista(listcod)
    numcod=contarcodones(mARNcod,listcod)
    totalaminoacido=totporaminoacido(amino,numcod)
    cuantosporaa=cuantosporaminoacido(amino)
    RSCU=calcularRSCU(numcod,totalaminoacido,cuantosporaa)
    porcentaje=calcporcentaje(numcod,totalaminoacido)
    tRNA=TRNA(genomas,listcod)
    tabla=creartabla(listcod,amino,numcod,RSCU,porcentaje,tRNA)
    return tabla
ID="NC_016736.1"
correo="evelas13@eafit.edu.co"
gencode=1
print(codonusagebias(ID,correo,gencode))

   Codon Amino acid Number   RSCU      %      tRNA
0    UAA          *   1859  1.225  40.85         0
1    GCG          A    366  0.592  14.79         0
2    AAG          K   1537  0.663  33.16         0
3    GUA          V   1120  1.216  30.41  trnV-UAC
4    AGA          R   1513  1.882  31.36  trnR-UCU
..   ...        ...    ...    ...    ...       ...
59   GAC          D    601  0.524  26.22  trnD-GUC
60   CUG          L    802  0.607  10.12         0
61   UUU          F   3347   1.19   59.5         0
62   ACA          T   1024  1.188   29.7  trnT-UGU
63   ACC          T    911  1.057  26.42  trnT-GGU

[64 rows x 6 columns]
